In [ ]:
%%capture
!pip install plotly

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

pd.set_option("display.max_columns", 100)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Street Smarts

## Data

#### Orignal data source found [here](https://www.fueleconomy.gov/feg/download.shtml) (fueleconomy.gov)

#### Data Dicitonary found [here](https://www.fueleconomy.gov/feg/ws/index.shtml#vehicle)

#### Fuel Economy guide for 2020 vehicles [here](https://www.fueleconomy.gov/feg/pdfs/guides/FEG2020.pdf)

#### Find the actual values used that create the EPA estimates for fuel mileage [here](https://www.fueleconomy.gov/feg/Find.do?action=bt1). Read the fin print on each label to find out how its calculated

## Evaluation Data (potential)

Even though a dedicated Co2 column doesnt exist until 2013, we can still have access to over [3 million](https://www.cars.com/for-sale/searchresults.action/?page=1&perPage=20&rd=99999&searchSource=GN_REFINEMENT&sort=relevance&yrId=47272%2C51683%2C56007%2C58487%2C30031936%2C35797618%2C36362520%2C36620293&zc=98118) listings using cars.com alone, creating a substantial evaluation set. Cars.com has an API found here https://developer.cars.com/apis

In [3]:
# Data from EPA includes all vehicles from 
# path = "../data/vehicles.csv"
path = "https://raw.githubusercontent.com/mpHarm88/streetsmart/master/data/vehicles.csv"
df = pd.read_csv(path)
print(f"The shape of the data: {df.shape}")
print(df.columns)

The shape of the data: (42230, 83)
Index(['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'guzzler',
       'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2', 'rangeA',
       'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c240bDsc

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
# Years included and their counts
df["year"].value_counts()

1984    1964
1985    1701
2018    1344
2019    1335
2017    1293
2015    1283
2016    1262
1987    1247
2014    1225
1986    1210
2020    1204
2008    1187
2013    1184
2009    1184
2005    1166
1989    1153
2012    1152
1991    1132
2011    1130
1988    1130
2007    1126
2004    1122
1992    1121
2010    1109
2006    1104
1993    1093
1990    1078
2003    1044
1994     982
2002     975
1995     967
2001     911
1999     852
2000     840
1998     812
1996     773
1997     762
2021      73
Name: year, dtype: int64

In [ ]:
df.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1300,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,2450,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-7250,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1000,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,0,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,2450,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-7250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2000,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-5000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [ ]:
df.tail()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
42225,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0,0.0,37.0,0.0,Compact Cars,1993,-1250,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
42226,14.330870,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1200,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,0.0,0,0,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,25.0,0.0,39.0,0.0,Compact Cars,1993,-1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
42227,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66030,(FFS),-1,1300,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,0.0,0.0,0.0,0,0,9997,0,14,Subaru,Legacy AWD,Y,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,23.0,0.0,34.0,0.0,Compact Cars,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
42228,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66030,(FFS),-1,1300,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,0.0,0.0,0.0,0,0,9998,0,14,Subaru,Legacy AWD,Y,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.0,0.0,34.0,0.0,Compact Cars,1993,-1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
42229,18.311667,0.0,0.0,0.0,16,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,493.722222,18,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2100,0,Premium,Premium Gasoline,-1,-1,21,0.0,0,0.0,0.0,0.0,0.0,0,0,9999,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,20.0,0.0,29.0,0.0,Compact Cars,1993,-5500,NaN,CLKUP,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [ ]:
# Find NaN
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42230 entries, 0 to 42229
Data columns (total 83 columns):
barrels08          42230 non-null float64
barrelsA08         42230 non-null float64
charge120          42230 non-null float64
charge240          42230 non-null float64
city08             42230 non-null int64
city08U            42230 non-null float64
cityA08            42230 non-null int64
cityA08U           42230 non-null float64
cityCD             42230 non-null float64
cityE              42230 non-null float64
cityUF             42230 non-null float64
co2                42230 non-null int64
co2A               42230 non-null int64
co2TailpipeAGpm    42230 non-null float64
co2TailpipeGpm     42230 non-null float64
comb08             42230 non-null int64
comb08U            42230 non-null float64
combA08            42230 non-null int64
combA08U           42230 non-null float64
combE              42230 non-null float64
combinedCD         42230 non-null float64
combinedUF         42

### Numerics

"co2" column looks to be filled with -1 values which im assuming was a placeholder for NaN. I'm assuming there were some years where CO2 was not recorded due to global warming not being a huge concern during those times (pre 2000)

In [ ]:
df.describe()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,engId,feScore,fuelCost08,fuelCostA08,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,UCity,UCityA,UHighway,UHighwayA,year,youSaveSpend,charge240b,phevCity,phevHwy,phevComb
count,42230.000000,42230.000000,42230.0,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,41990.000000,41992.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000,42230.000000
mean,17.208441,0.221610,0.0,0.054709,18.512929,6.441259,0.726450,0.584693,0.000450,0.397819,0.002062,95.620791,5.961804,17.551395,464.279643,20.757755,7.173490,0.784608,0.620800,0.404657,0.000347,0.002042,5.713503,3.289591,7963.338054,0.474473,1601.633910,96.807956,0.473692,-0.920554,24.641748,8.441684,0.886834,0.690087,0.000232,0.413245,0.002015,1.992375,10.183614,21238.705115,1.794790,6.092351,13.461591,33.772839,0.978025,0.957081,0.113151,0.909845,0.105257,23.410311,0.948664,34.555428,1.228918,2002.011414,-3005.209567,0.012583,0.191309,0.192896,0.191191
std,4.649507,1.142513,0.0,0.665389,8.355167,12.181091,5.657377,5.527820,0.038272,4.268703,0.033192,180.085172,57.549604,93.436211,124.384072,8.062297,12.826143,5.668628,5.492966,4.298840,0.033535,0.032815,1.758066,1.356875,17082.701784,2.776665,482.583847,509.667828,2.777604,0.658698,8.026049,14.294617,5.851538,5.606418,0.028039,4.354442,0.032346,5.921842,27.940778,12319.604224,4.348795,9.605151,31.016657,46.048681,14.874041,15.006130,2.294075,14.549553,2.049242,11.881505,7.954267,11.664992,8.332576,11.382099,2414.683502,0.278699,3.235159,3.127795,3.166686
min,0.060000,0.000000,0.0,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,-1.000000,450.000000,0.000000,-1.000000,-1.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1984.000000,-22000.000000,0.000000,0.000000,0.000000,0.000000
25%,14.330870,0.000000,0.0,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,386.391304,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.200000,0.000000,-1.000000,1250.000000,0.000000,-1.000000,-1.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10558.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.300000,0.000000,28.000000,0.000000,1991.000000,-4500.000000,0.000000,0.000000,0.000000,0.000000
50%,16.480500,0.000000,0.0,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,444.350000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,3.000000,172.000000,-1.000000,1500.000000,0.000000,-1.000000,-1.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21117.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.500000,0.000000,33.333300,0.000000,2003.00

### Categoricals

In [ ]:
df.describe(exclude="number")

,drive,eng_dscr,fuelType,fuelType1,make,model,mpgData,phevBlended,trany,VClass,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,c240bDscr,createdOn,modifiedOn,startStop
count,41041,25898,42230,42230,42230,42230,42230,42230,42219,42230,2483,15047,7442,878,3762,1673,1668,1009,11412,101,95,42230,42230,10526
unique,7,560,14,6,137,4217,2,2,38,34,3,52,1,1,8,4,231,170,49,5,7,312,176,2
top,Front-Wheel Drive,(FFS),Regular,Regular Gasoline,Chevrolet,F150 Pickup 2WD,N,False,Automatic 4-spd,Compact Cars,G,CLKUP,T,S,FFV,E85,290,288V Ni-MH,GMX,standard charger,80 amp dual charger,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,N
freq,14515,8827,26913,28335,4086,224,28934,42104,11048,5988,1504,7809,7442,878,1466,1466,74,131,1639,90,55,34217,29343,6675


### Find where co2 feature equals -1, and how many

Seeing that there are ~31,000 columns where co2 equals one, it becomes a problem if we are able to find out how to calculate co2 for these older vehicles

In [ ]:
# Rows where co2 equals -1
df[df["co2"] == -1].shape

(31954, 83)

In [ ]:
# Rows where data doesnt equal -1
df[df["co2"] != -1].shape

(10276, 83)

In [ ]:
# Are these Electric Vehicles?
df[df["co2"] == 0].shape

(237, 83)

In [ ]:
# wrangle

def cleaner(df):
    df = df.copy()
    
    
    
    return df

In [ ]:
df.columns

Index(['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'guzzler',
       'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2', 'rangeA',
       'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c240bDscr',
       'createdOn', 'modifiedOn

## Subsetting electric vehicles only and their variants

In [9]:
#Subset rows where evmotor is not False
df_ev = df[~df["evMotor"].isnull()]

# Pick out desirable columns

ls = ['make', 'model', 'year', 'charge240', 'city08','city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2','co2A', 
      'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U','combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF',
      'feScore', 'fuelCost08', 'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA','highway08', 'highway08U', 
      'highwayA08', 'highwayA08U', 'highwayCD','highwayE', 'highwayUF', 'mpgData', 'phevBlended', 'rangeCityA', 'rangeHwy', 'rangeHwyA', 
      'trany', 'UCity', 'UCityA', 'UHighway', 'UHighwayA', 'VClass', 'youSaveSpend', 'guzzler', 'tCharger', 'sCharger', 'atvType', 'fuelType2', 'rangeA',
       'evMotor', 'c240Dscr', 'charge240b', 'c240bDscr', 'startStop', 'phevCity', 'phevHwy',
       'phevComb']

df_ev[ls].head()

,make,model,year,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,mpgData,phevBlended,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,youSaveSpend,guzzler,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,c240Dscr,charge240b,c240bDscr,startStop,phevCity,phevHwy,phevComb
7138,Nissan,Altra EV,2000,0.0,81,0.0,0,0.0,0.0,41.0,0.0,0,-1,0.0,0.0,85,0.0,0,0.0,40.0,0.0,0.0,-1,800,0,Electricity,Electricity,-1,-1,91,0.0,0,0.0,0.0,37.0,0.0,N,False,0.0,0.0,0.0,NaN,116.2069,0.0,129.6154,0.0,Midsize Station Wagons,1000,NaN,NaN,NaN,EV,NaN,NaN,62 KW AC Induction,NaN,0.0,NaN,N,0,0,0
7139,Toyota,RAV4 EV,2000,0.0,81,0.0,0,0.0,0.0,41.0,0.0,0,-1,0.0,0.0,72,0.0,0,0.0,47.0,0.0,0.0,-1,900,0,Electricity,Electricity,-1,-1,64,0.0,0,0.0,0.0,53.0,0.0,N,False,0.0,0.0,0.0,NaN,116.2069,0.0,91.0811,0.0,Sport Utility Vehicle - 2WD,500,NaN,NaN,NaN,EV,NaN,NaN,50 KW DC,NaN,0.0,NaN,N,0,0,0
8143,Toyota,RAV4 EV,2001,0.0,81,0.0,0,0.0,0.0,41.0,0.0,0,-1,0.0,0.0,72,0.0,0,0.0,47.0,0.0,0.0,-1,900,0,Electricity,Electricity,-1,-1,64,0.0,0,0.0,0.0,53.0,0.0,N,False,0.0,0.0,0.0,NaN,116.2069,0.0,91.0811,0.0,Sport Utility Vehicle - 2WD,500,NaN,NaN,NaN,EV,NaN,NaN,50 KW DC,NaN,0.0,NaN,N,0,0,0
8144,Ford,Th!nk,2001,0.0,74,0.0,0,0.0,0.0,46.0,0.0,0,-1,0.0,0.0,65,0.0,0,0.0,52.0,0.0,0.0,-1,1000,0,Electricity,Electricity,-1,-1,58,0.0,0,0.0,0.0,59.0,0.0,N,False,0.0,0.0,0.0,NaN,105.3125,0.0,82.1951,0.0,Two Seaters,0,NaN,NaN,NaN,EV,NaN,NaN,27 KW AC Induction,NaN,0.0,NaN,N,0,0,0
8146,Ford,Explorer USPS Electric,2001,0.0,45,0.0,0,0.0,0.0,75.0,0.0,0,-1,0.0,0.0,39,0.0,0,0.0,87.0,0.0,0.0,-1,1700,0,Electricity,Electricity,-1,-1,33,0.0,0,0.0,0.0,102.0,0.0,N,False,0.0,0.0,0.0,NaN,62.4074,0.0,46.8056,0.0,Sport Utility Vehicle - 2WD,-3500,NaN,NaN,NaN,EV,NaN,NaN,67 KW AC Induction,NaN,0.0,NaN,N,0,0,0


### Plotly

Docs: [here](https://plotly.com/python-api-reference/plotly.express.html)

In [57]:
#Subset data to only major manufacturers
choice = ["Ford", "Nissan", "Toyota", "Chevrolet"]
df_sub = df[df["make"].isin(choice)]

df_sub.shape

(11155, 83)

In [62]:
px.scatter(df_sub, 
           x="co2",
           y="comb08",
           color="fuelType1",
           hover_name="model", 
           hover_data=["evMotor", "guzzler", "combE"], 
           title="Does higher co2 level ever indicate better MPG?"
           )

In [ ]:
px.violin(df_sub,
          x="make", 
          y="comb08", 
          hover_data=["evMotor", "guzzler", "combE", "year"],
          hover_name="model",
          title="Visualizing the distribution of manufactuer models and their combined MPG")